In [148]:
import numpy as np
from collections import defaultdict
import random

In [2]:
DAYS = {"M": "000", "T": "001", "W": "010", "TH": "011", "F":"100"}

In [3]:
DAYS_INV = dict((v, k) for k, v in DAYS.items())

In [4]:
TIMES = {8: "0000", 9: "0001", 10: "0010", 11: "0011", 12: "0100", 13: "0101", 14: "0110", 15: "0111", 16: "1000"}

In [5]:
TIMES_INV = dict((v, k) for k, v in TIMES.items())

In [6]:
LENGTHS = {1: "00", 1.5: "01", 2: "10"}

In [7]:
LENGTHS_INV = dict((v, k) for k, v in LENGTHS.items())

In [8]:
ROOMS = {"R1": "0000", "R2": "0001", "R3": "0010", "R4": "0011", "R5": "0100", "R6": "0101", "R7": "0110", "R8": "0111", "R9": "1000"}

In [9]:
TEACHER = {"T1": "0000", "T2": "0001", "T3": "0010", "T4": "0011", "T5": "0100", "T6": "0101", "T7": "0110", "T8": "0111", "T9": "1000"}

In [10]:
CLASS = {"C1": "0000", "C2": "0001", "C3": "0010", "C4": "0011", "C5": "0100", "C6": "0101", "C7": "0110", "C8": "0111", "C9": "1000"}

In [103]:
DICTS = {"DAYS": DAYS, "TIMES": TIMES, "LENGTHS": LENGTHS, "ROOMS": ROOMS, "TEACHER": TEACHER, "CLASS": CLASS,}

In [104]:
GENE_LENGTHS = {"DAYS": len(DAYS["M"]), "TIMES": len(TIMES[8]), "LENGTHS": len(LENGTHS[1]), "ROOMS": len(ROOMS["R1"]), "TEACHER": len(TEACHER["T1"]),
                "CLASS": len(CLASS["C1"]),}

Starts at 8 for 2.0 hours on M
Starts at 8 for 2.0 hours on T
Starts at 8 for 2.0 hours on W
Starts at 8 for 2.0 hours on TH
Starts at 8 for 2.0 hours on F


In [24]:
def get_gene_bounds(GENE_LENGTHS):
    bounds = {}
    start = 0
    for key in GENE_LENGTHS.keys():
        end = GENE_LENGTHS[key] + start
        bounds[key] = (start, end)
        start += GENE_LENGTHS[key]
    return bounds

BOUNDS = get_gene_bounds(GENE_LENGTHS)        

In [43]:
class BinFuncs:
    @staticmethod
    def get_slot(s1):
        time_len = GENE_LENGTHS["TIMES"]
        day_len = GENE_LENGTHS["DAYS"]
        length_len = GENE_LENGTHS["LENGTHS"]
        
        d_s, d_e = BOUNDS["DAYS"]
        t_s, t_e = BOUNDS["TIMES"]
        l_s, l_e = BOUNDS["LENGTHS"]
        
        day_1 = DAYS_INV[s1[d_s:d_e]]
        time_1 = TIMES_INV[s1[t_s:t_e]]
        length_1 = LENGTHS_INV[s1[l_s:l_e]]
        
        return (day_1, time_1, length_1)
    
    @staticmethod
    def is_overlapping(tuple1, tuple2):
        day_1, time_1, length_1 = tuple1
        day_2, time_2, length_2 = tuple2
        return Event(day_1, time_1, length_1).is_overlapping(Event(day_2, time_2, length_2))        

In [255]:
import random
class GA:
    

#     def generate_pop(size=10):
#         pop_list = []
#         for i in range(size):
            
    
    # Swap the genes of two parents
    @staticmethod
    def swap(p1, p2, attr):              
        p1 = list(p1)
        p2 = list(p2)

        start_index, end_index = BOUNDS[attr]

        p1[start_index:end_index], p2[start_index:end_index] = p2[start_index:end_index], p1[start_index:end_index]

        return ''.join(p1) , ''.join(p2)
    
    # Get a random mutation on time attr
    @staticmethod
    def mutate_time(p1):
        p1 = list(p1)
        
        keys_list = list(BOUNDS.keys())[:2]
        
        attr = random.choice(keys_list)
        new_val = random.choice(list(DICTS[attr].values()))
        start, end = BOUNDS[attr]
        
        p1[start:end] = new_val
        
        return ''.join(p1)
    
    @staticmethod
    def inverse_fitness(s_bin, ITEM_LEN):
        total_conflicts = 0
        days = defaultdict(list)
        index = 0

        for i in range(ITEM_LEN, len(s_bin), ITEM_LEN):
            event_one = BinFuncs.get_slot(s_bin[i:i+ITEM_LEN])
            for j in range(i-ITEM_LEN, -1, -ITEM_LEN):
                event_two = BinFuncs.get_slot(s_bin[j:j+ITEM_LEN])
                conflict = BinFuncs.is_overlapping(event_one, event_two)
                if conflict:
                    total_conflicts += 1
#                     print("conflict")
                    


        return 1./(1. + total_conflicts)
    
    @staticmethod
    def time_prefs(s_bin, time_prefs, ITEM_LEN):
        PREF = 0.05
        penalty = 0
        
        for i in range(ITEM_LEN, len(s_bin), ITEM_LEN):
            event_one = BinFuncs.get_slot(s_bin[i:i+ITEM_LEN])
            for pref_event in (time_prefs):
                conflict = BinFuncs.is_overlapping(event_one, BinFuncs.get_slot(pref_event.get_genome()))
                if conflict:
                    penalty -= PREF
                    break
                    
        return penalty
    
    # Blending two parents and producing two children.
    # https://www.researchgate.net/publication/282269189_An_empirical_comparison_of_two_crossover_operators_in_real-coded_genetic_algorithms_for_constrained_numerical_optimization_problems
    
    @staticmethod
    def blend_crossover(x1, x2):
        min_start = min(x1.start, x2.start)
        max_end = max(x1.start + x1.len, x2.start + x2.len)
        
        diff = max_end-min_start
        alpha = 0.5
        
        i_start = round(min_start - (diff * alpha))
        i_end = round(max_end + (diff * alpha))
                
        # This part should prob assign intervals bounds to key with least difference
        # instead of current implementation
        if(i_start not in TIMES.keys()):
            i_start = min(list(TIMES.keys()))
        
        if(i_end not in TIMES.keys()):
            i_end = max(list(TIMES.keys()))
            
        parent_time_range = range(i_start, i_end, 1)
        
        time_one = random.choice(parent_time_range)
        time_two = random.choice(parent_time_range)
        
        parent_day_range = [x1.day, x2.day]
        
        day_one = random.choice(parent_day_range)
        day_two = random.choice(parent_day_range)
        
        child_1 = Event(day_one, time_one, x1.len)
        child_2 = Event(day_two, time_two, x2.len)
        
        return (child_1, child_2)
        
        
        
        
#     def algorithm(initial_pop, generations=10):
#         current_pop = initial_pop
#         best_pop = initial_pop
#         for i in range(generations):
#             if inverse_fitness(current_pop) > inverse_fitness(best_pop)
#                 best_pop = current_pop
#             for schedule in current_population:
#                 # For best two individual schedules
#                 for event in schedule:
                    
                
        

In [270]:
d1 = Event("M", 8, 2)
d2 = Event("W", 9, 2)

c1, c2 = GA.blend_crossover(d1, d2)
print(c1, c2)

Starts at 9 for 2.0 hours on W Starts at 9 for 2.0 hours on W


In [269]:
    #     print(days)

    #     for key in days.keys():
    #         events = days[key]

    #         for i in range(1, len(events)):
    #             event_one = BinFuncs.get_slot(key+events[i])
    #             print(f"Outer {event_one} index {i}")

    #             for j in range(i-1, -1, -1):
    #                 event_two = BinFuncs.get_slot(key+events[j])
    #                 print(f"Inner {event_two} index {j}")
    #                 conflict = BinFuncs.is_overlapping(event_one, event_two)
    #                 if conflict:
    #                     print("conflict")
    #                     total_conflicts += 1
    #     for key in days.keys():
    #         days[key] = sorted(days[key])

    #         ongoing = 0
    #         for event in days[key]:


    #     for event s.schedule:
    #         print(eventOne)
    #         for eventTwo in s.schedule:
    #             if eventOne != eventTwo:
    #                 if eventOne.is_overlapping(eventTwo):
    #                     total_conflicts+= 1

In [177]:
p1 = Event("M", 11, 1)
print(p1)
d, t, l = BinFuncs.get_slot(GA.mutate_time(p1.get_genome()))
print(Event(d, t, l))
d, t, l = BinFuncs.get_slot(GA.mutate_time(p1.get_genome()))
print(Event(d, t, l))
d, t, l = BinFuncs.get_slot(GA.mutate_time(p1.get_genome()))
print(Event(d, t, l))

Starts at 11 for 1.0 hours on M
Starts at 8 for 1.0 hours on M
Starts at 11 for 1.0 hours on TH
Starts at 15 for 1.0 hours on M


In [219]:
p1 = Event("M", 11, 1)
p1.teacher = "T2"
p2 = Event("T", 10, 1)

p3 = Event("W", 12, 2)
print(p1.get_genome(), p2.get_genome())
p1_cross, p2_cross = GA.swap(p1.get_genome(), p2.get_genome(), "DAYS")
print(p1_cross, p2_cross)
# print(LENGTHS[1.5])
# print(LENGTHS[1])

000001100000000010000 001001000000000000000
001001100000000010000 000001000000000000000


In [179]:
class Event:
    def __init__(self, day, start, length):
        self.start = int(start)
        self.len = float(length)
        self.day = day
        
        self.teacher = "T1"
        self.class_name = "C1"
        self.room = "R1"
        
    def is_overlapping(self, event):
        end = self.start + self.len
        other_end = event.start + event.len
        
        end_during = end > event.start and end <= other_end
        start_during = self.start >= event.start and self.start < other_end
        same_day = (self.day == event.day)
        
        return (end_during or start_during) and same_day

    
    def __str__(self):
        return f"Starts at {self.start} for {self.len} hours on {self.day}"
    
    def time_gene(self):
        return DAYS[self.day] + TIMES[self.start] + LENGTHS[self.len]
    
    def get_genome(self):
        genome = ""
        genome += self.time_gene()
        genome += ROOMS[self.room]
        genome += TEACHER[self.teacher]
        genome += CLASS[self.class_name]
        return genome

In [114]:
# print(Event(15, 2.6, "M").is_overlapping(Event(12, 3.1, "W")))

In [159]:
def random_start_time():
    days = list(DAYS.keys())
    times = list(TIMES.keys())
    length = 2
    
    return random.choice(days), random.choice(times), 2

In [205]:
class Schedule:
    def __init__(self, events = 10):
        self.schedule = []
                
        for x in range(events):
            # Get random 
            d, t, l = random_start_time()
            self.schedule.append(Event(d, t, l))            
            
    def __str__(self):
        rep = ""
        for event in self.schedule:
            rep = rep + str(event) + " "
            rep = rep + "\n"
        return rep
    
    def print_genome(self):
        genome = ""
        for event in self.schedule:
            print(event.get_genome())
            
    def get_as_bin(self):
        genome = ""
        for event in self.schedule:
            genome += event.get_genome()
        return genome

In [161]:
# Rough implementation, doesn't really work yet


In [162]:
len(s.schedule[0].get_genome())

KeyError: '010'

In [246]:
s = Schedule(events=10)
s_2 = Schedule(events=10)
s_3 = Schedule(events=10)
# s.schedule = []
# s.schedule.append(Event("M", 12, 2))
# s.schedule.append(Event("F", 12, 2))
# s.schedule.append(Event("M", 8, 1))
# s.schedule.append(Event("M",8, 2))
# s.schedule.append(Event("T",8, 2))

# s.schedule.append(Event("T",8, 2))
# s.schedule.append(Event("T",8, 2))
print(GA.inverse_fitness(s.get_as_bin(), len(s.schedule[0].get_genome())))
empty_morns = [Event(x, 8, 2) for x in DAYS.keys()]
print(GA.time_prefs(s.get_as_bin(), empty_morns,len(s.schedule[0].get_genome())))
      
# print(GA.inverse_fitness(s_2.get_as_bin(), len(s.schedule[0].get_genome())))
# print(GA.inverse_fitness(s_3.get_as_bin(), len(s.schedule[0].get_genome())))

print(s)
# print(s_2)
# print(s_3)


0.25
-0.1
Starts at 14 for 2.0 hours on T 
Starts at 8 for 2.0 hours on M 
Starts at 12 for 2.0 hours on TH 
Starts at 8 for 2.0 hours on M 
Starts at 10 for 2.0 hours on W 
Starts at 14 for 2.0 hours on M 
Starts at 12 for 2.0 hours on W 
Starts at 12 for 2.0 hours on F 
Starts at 13 for 2.0 hours on W 
Starts at 14 for 2.0 hours on T 

